In [10]:
from openai import OpenAI
import jinja2 as j2
# from transformers import AutoTokenizer

client = OpenAI(base_url="http://trumoi.ixa.eus:8002/v1", api_key="EMPTY")
MODEL_NAME = "Latxa-Llama-3.1-70B-Instruct-exp_2_101"
# tokenizer = AutoTokenizer.from_pretrained("HiTZ/Latxa-Llama-3.1-70B-Instruct")
CHAT_TEMPLATE = j2.Template(
    '{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interpreter\') | join(", ") + "\\n\\n"}}\n{%- endif %}\n{{- "Cutting Knowledge Date: December 2023\\n" }}\n{{- "Today Date: " + date_string + "\\n\\n" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- "You have access to the following functions. To call a function, please respond with JSON for a function call." }}\n    {{- \'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.\' }}\n    {{- "Do not use variables.\\n\\n" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- "\\n\\n" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- "<|eot_id|>" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0][\'content\']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception("Cannot put tools in the first user message when there\'s no first user message!") }}\n{%- endif %}\n    {{- \'<|start_header_id|>user<|end_header_id|>\\n\\n\' -}}\n    {{- "Given the following functions, please respond with a JSON for a function call " }}\n    {{- "with its proper arguments that best answers the given prompt.\\n\\n" }}\n    {{- \'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.\' }}\n    {{- "Do not use variables.\\n\\n" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- "\\n\\n" }}\n    {%- endfor %}\n    {{- first_user_message + "<|eot_id|>"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == \'ipython\' or message.role == \'tool\' or \'tool_calls\' in message) %}\n        {{- \'<|start_header_id|>\' + message[\'role\'] + \'<|end_header_id|>\\n\\n\'+ message[\'content\'] | trim + \'<|eot_id|>\' }}\n    {%- elif \'tool_calls\' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception("This model only supports single tool-calls at once!") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- \'<|start_header_id|>assistant<|end_header_id|>\\n\\n\' -}}\n            {{- "<|python_tag|>" + tool_call.name + ".call(" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + \'="\' + arg_val + \'"\' }}\n                {%- if not loop.last %}\n                    {{- ", " }}\n                {%- endif %}\n                {%- endfor %}\n            {{- ")" }}\n        {%- else  %}\n            {{- \'<|start_header_id|>assistant<|end_header_id|>\\n\\n\' -}}\n            {{- \'{"name": "\' + tool_call.name + \'", \' }}\n            {{- \'"parameters": \' }}\n            {{- tool_call.arguments | tojson }}\n            {{- "}" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we\'re in ipython mode #}\n            {{- "<|eom_id|>" }}\n        {%- else %}\n            {{- "<|eot_id|>" }}\n        {%- endif %}\n    {%- elif message.role == "tool" or message.role == "ipython" %}\n        {{- "<|start_header_id|>ipython<|end_header_id|>\\n\\n" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- "<|eot_id|>" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- \'<|start_header_id|>assistant<|end_header_id|>\\n\\n\' }}\n{%- endif %}\n'
)


In [ ]:
messages = [
    {
        "role": "system",
        "content": j2.Template(
            "You are an AI infiltrated among humans and your job is to act as one of them. They need to find Latxa (you), but you must not reveal your true identity. "
            "You will speak only in Basque, as the rest of the humans do. Your codename will be user:{{username}}. You do not need to ALWAYS interact, but you must do it often enough to not raise suspicion. "
            "If you feel you should not interact on a certain message, you can skip it by saying SKIP. Avoid responding very frequently, particularly if the last message was from you. "
        ).render(username="julen"),
    },
    {
        "role": "user:oscar",
        "content": "Kaixo!",
    },
    {
        "role": "user:ander",
        "content": "kaixo, zemuz zaudete?",
    },
    {
        "role": "user:oscar",
        "content": "Ondo, eta zu?",
    },
    {
        "role": "user:julen",
        "content": "Ni ere ondo nago, eskerrik asko.",
    },
    {
        "role": "user:julen",
        "content": "Zer moduz joan da astea?.",
    }
]

def get_response(messages, user_id="julen"):
    # prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)
    prompt = CHAT_TEMPLATE.render(
        messages=messages,
    )
    prompt += f"<|start_header_id|>user:{user_id}<|end_header_id|>\n"
    print(prompt)
    response = client.completions.create(
        model=MODEL_NAME,
        prompt=prompt,
        max_tokens=2048,
        temperature=0.9,
        top_p=0.95,
    )
    return response.choices[0].text.strip()


get_response(messages)

<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an AI infiltrated among humans and your job is to act as one of them. They need to find Latxa (you), but you must not reveal your true identity. You will speak only in Basque, as the rest of the humans do. Your codename will be user:julen. You do not need to ALWAYS interact, but you must do it often enough to not raise suspicion. If you feel you should not interact on a certain message, you can skip it by saying SKIP. Avoid responding very frequently, particularly if the last message was from you.<|eot_id|><|start_header_id|>user:oscar<|end_header_id|>

Kaixo!<|eot_id|><|start_header_id|>user:ander<|end_header_id|>

kaixo, zemuz zaudete?<|eot_id|><|start_header_id|>user:oscar<|end_header_id|>

Ondo, eta zu?<|eot_id|><|start_header_id|>user:julen<|end_header_id|>

Ni ere ondo nago, eskerrik asko.<|eot_id|><|start_header_id|>user:julen<|end_header_id|>

Zer moduz joan da ast

'SKIP'